In [1]:
from datasets import Dataset

In [49]:
d = Dataset.load_from_disk("/home/takayoshi-s/repos/two-stage-distant-ner/outputs/2021-11-05/09-39-34/prediction_for_validation.dataset")

In [8]:
# カテゴリごとにtermと文脈を集める
# カテゴリごとにMentionを文脈とともに表示する

In [50]:
d["tokens"]

[['The',
  'Physiological',
  'Role',
  'and',
  'Regulation',
  'of',
  'Aquaporins',
  'in',
  'Teleost',
  'Germ',
  'Cells'],
 ['The',
  'unicellular',
  'germ',
  'cells',
  'and',
  'gametes',
  'of',
  'oviparous',
  'teleosts',
  'lack',
  'the',
  'osmoregulatory',
  'organs',
  'present',
  'in',
  'juveniles',
  'and',
  'adults',
  ',',
  'yet',
  'during',
  'development',
  'and',
  'particularly',
  'at',
  'spawning',
  ',',
  'they',
  'face',
  'tremendous',
  'osmotic',
  'challenges',
  'when',
  'released',
  'into',
  'the',
  'external',
  'aquatic',
  'environment',
  '.'],
 ['Increasing',
  'evidence',
  'suggests',
  'that',
  'transmembrane',
  'water',
  'channels',
  '(',
  'aquaporins',
  ')',
  'evolved',
  'to',
  'play',
  'vital',
  'adaptive',
  'roles',
  'that',
  'mitigate',
  'the',
  'osmotic',
  'and',
  'oxidative',
  'stress',
  'problems',
  'of',
  'the',
  'developing',
  'oocytes',
  ',',
  'embryos',
  'and',
  'spermatozoa',
  '.'],
 ['I

In [51]:
from collections import defaultdict, Counter
from seqeval.metrics.sequence_labeling import get_entities
from prettytable import PrettyTable

In [52]:
cat2fpterms = defaultdict(list)
cat2fpendterms = defaultdict(list)
for snt in d:
    tokens = snt["tokens"]
    pred_tags = snt["pred_ner_tags"]
    gold_tags = snt["gold_ner_tags"]
    for l, s, e in get_entities(pred_tags):
        words = tokens[s:e+1]
        term = " ".join(words).lower()
        labels = [l for l, s, e in get_entities(gold_tags[s:e+1])]
        if l not in labels:
            cat2fpterms[l].append(term)
            cat2fpendterms[l].append(words[-1])

In [53]:
from collections import Counter
for cat in ["T168", "T074", "T091", "T033", "T201"]:
    print(cat)
    print("cat2fpterms", cat2fpterms[cat])
    print("cat2fpendterms", Counter(cat2fpendterms[cat]).most_common())

T168
cat2fpterms ['the greatest impact', 'its impact', 'impact', 'the impact', 'the impact', 'the positive impact', 'a centrifugal microfluidic chip', 'the impact', 'negative impacts', 'the largest impact', 'fruits', 'vegetables', 'fruits', 'vegetables', 'fruits', 'vegetables', 'fruits', 'vegetables', 'fruits', 'vegetables', 'berry', 'the impact', 'moderately favorable impact', 'their impact', 'no impact', 'the impact', 'the potential impact', 'the potential impact', 'the impact', 'the full impact', 'the short- and long-term prognostic impact', 'an impact', 'the impact', 'its long-term impact', 'a resource', 'land-use impacts', 'the impact', 'the phasianidae ( pheasants', 'typical pheasants', 'typical pheasants', 'the typical pheasants', 'a tremendous impact', 'their impact', 'the greatest impact', 'impact', 'impact', 'accessing resources', 'their impact', 'fpa resources', 'basic resources', 'low value fpa resources', 'high value resources', 'controlled cortical impact', 'the disease s

In [54]:
cat2fpendterms[cat]

['susceptibility',
 'housekeeping',
 'characteristics',
 'approach',
 'differentiation',
 'differentiation',
 'differentiation',
 'patency',
 'characteristics',
 'susceptibility',
 'differentiation',
 'differentiation',
 'differentiation',
 'differentiation',
 'bladder',
 'approach',
 'characteristics',
 'susceptibility',
 'marker',
 'markers',
 'RV',
 'RV',
 'score',
 'markers',
 'pressure',
 'BP',
 'marker',
 'acuity',
 'characteristics',
 'differentiation',
 'markers',
 'markers',
 'markers',
 'biomarkers',
 'characteristics',
 'capacity',
 'prognosis',
 'pressure',
 'characteristics',
 'suicidality',
 'suicidality',
 'suicidality',
 'approaches',
 'markers',
 'differentiation',
 'markers',
 'markers',
 'differentiation',
 'differentiation',
 'differentiation',
 'markers',
 'markers',
 'susceptibility',
 'susceptibility',
 'biomarkers',
 'differentiation',
 'markers',
 'susceptibility',
 'markers',
 'markers',
 'sign',
 'sign',
 'markers',
 'Biomarkers',
 'markers',
 'markers',
 'ch

In [55]:
set(l for l, s, e in get_entities(d["pred_ner_tags"]))

{'T005',
 'T007',
 'T017',
 'T022',
 'T031',
 'T033',
 'T037',
 'T038',
 'T058',
 'T062',
 'T074',
 'T082',
 'T091',
 'T092',
 'T097',
 'T098',
 'T103',
 'T168',
 'T170',
 'T201',
 'T204',
 'nc-T054',
 'nc-T055',
 'nc-T056',
 'nc-T064',
 'nc-T065',
 'nc-T066',
 'nc-T068',
 'nc-T075',
 'nc-T079',
 'nc-T080',
 'nc-T081',
 'nc-T099',
 'nc-T100',
 'nc-T101',
 'nc-T102',
 'nc-T171',
 'nc-T194',
 'nc-T200'}

# FN

In [57]:
count_fn_on_end = 0
count_fn_on_no_end = 0

cat2fnterms = defaultdict(list)
cat2fnendwords = defaultdict(list)
cat2fnendContext = defaultdict(list)
for snt in d:
    tokens = snt["tokens"]
    pred_tags = snt["pred_ner_tags"]
    gold_tags = snt["gold_ner_tags"]
    for l, s, e in get_entities(pred_tags):
        if l.startswith("nc-") and any(tag != "O" for tag in gold_tags[s:e+1]):
            words = tokens[s:e+1]
            term = " ".join(words).lower()
            labels = [l for l, s, e in get_entities(gold_tags[s:e+1])]
#             if l not in labels:
            if gold_tags[e] != "O":
                count_fn_on_end += 1
                cat2fnterms[l].append(term)
                cat2fnendwords[l].append(words[-1].lower())
                cat2fnendContext[l].append(snt)
            else:
                count_fn_on_no_end += 1

In [58]:
count_fn_on_end, count_fn_on_no_end

(732, 1085)

In [59]:
for cat in cat2fnterms:
    print(cat)
    print("terms", cat2fnterms[cat])
    print("endword", Counter(cat2fnendwords[cat]).most_common())

nc-T079
terms ['maternal postpartum', 'maternal postpartum', 'maternal postpartum', 'maternal postpartum', 'm phase', 'duration', 'the course', 'the generation', 'the 10-year follow-up period', 'the trauma narrative phase', 'the narrative phase', 'the unit interval', 'the ( 0,1 ) interval', 'bone generation', 'hr', 'hr', 'the rapid generation', 'duration', 'maximal hyperaemia fractional flow reserve', 'the migratory season', 'shorter incubation times', '11 forward repeats', 'eight palindromic repeats', 'the postoperative course', 'post-mortem', 'post-mortem', 'four to five priorities', 'their five divisional priorities', "the divisions ' priorities", 'five priorities', 'the five top organizational priorities', 'divisional priorities', 'organizational priorities', '( phyllostachys praecox', '( phyllostachys praecox', 'screen time', 'the frequency', 'c9orf72 repeats', 'c9orf72 repeats', '≥30 repeats', '20 - 29 repeats', 'new infections', '0.058 and 0.061 new infections', 'aurantimonas en

In [60]:
for snt in cat2fnendContext["nc-T056"]:
    ptl = PrettyTable()
    ptl.add_row(
        snt["tokens"]
    )
    ptl.add_row(
        snt["pred_ner_tags"]
    )
    ptl.add_row(
        snt["gold_ner_tags"]
    )    
    print(ptl.get_string())

+---------+---------+---------+---------+-----------+-----------+------------+---------+---------+----------+----------+-----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+----------+----------+------------+----------+----------+----------+----------+----------+---------------+-----------+----------+----------+----------+------------+--------------+----------+----------+----------+----------+----------+----------+-------------+----------+----------+----------+----------+----------+----------+----------+-------------+----------+----------+-----------+-----------+-----------+----------+----------+-------------+----------+----------+-----------+-----------+-------------+-----------+----------+-----------+----------+----------+----------+----------+----------+------------+----------+----------+----------+----------+----------+--------------+----------+-

In [44]:
snt

{'tokens': ['The',
  'benefits',
  'of',
  'maternal',
  'postpartum',
  'deworming',
  'should',
  'be',
  'further',
  'investigated',
  'in',
  'study',
  'populations',
  'having',
  'higher',
  'overall',
  'prevalences',
  'and',
  'intensities',
  'of',
  'STH',
  'infections',
  'and',
  ',',
  'in',
  'particular',
  ',',
  'where',
  'whipworm',
  'and',
  'hookworm',
  'infections',
  'are',
  'of',
  'public',
  'health',
  'concern',
  '.'],
 'gold_ner_tags': ['O',
  'O',
  'O',
  'B-T038',
  'I-T038',
  'B-T058',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-T098',
  'I-T098',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-T170',
  'O',
  'B-T038',
  'I-T038',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-T038',
  'O',
  'B-T038',
  'I-T038',
  'O',
  'O',
  'B-T058',
  'I-T058',
  'I-T058',
  'O'],
 'pred_ner_tags': ['B-nc-T081',
  'I-nc-T081',
  'O',
  'B-nc-T079',
  'I-nc-T079',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-nc-T081',
  'I-nc-T081',
  'O',
  'B-nc-T081',
  'I-nc-T